# Modeling Template Notebook - v1

This notebook should serve as a template for the standard modelling we will all be using for as the base of version 1 development.

The goal is for us to use the same predicted probabilities and classes to build different functions according the to tasks assigned to us.

The notebook should be used as follows:
1. Import all needed packages:
2. Load Data
3. Split Train and Test Sets
4. Build and Run model
5. Output predictions

### Import necessary packages

Note: In the case a Module Not Found error is thrown, it will automatically attempt to install all needed packages. You would have to rerun the import package cell again after.

In [2]:
def install_packages():
    !pip install pandas
    !pip install numpy
    !pip install scikit-learn

In [44]:
try:
    import pandas as pd
    import numpy as np
    from sklearn import datasets
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import precision_score, recall_score, precision_recall_curve
    print('import successful')
except ModuleNotFoundError:
    print('forcing install of necessary packages. If you see this, rerun this cell to try again')
    install_packages()

import successful


### Load necessary data

The dataset we're starting off with is the standard breast cancer dataset that lends itself directly to binary classification modelling.

In [3]:
X, y = datasets.load_breast_cancer(return_X_y=True)

### Train Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Run model

We're using the simplest flavour of the ordinary least squares logistic regression in this case and no data transformation / scaling to maintain simplicity

In [5]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

### Output Predictions

This section provides the basic outputs we will be using for function building as variables. They are as follows:
1. predicted_probabilities: this is output of pre for which we will be performing optizations. This has two 'columns' where the values in index 0 are the probabilities for the class 0 and those in index 1 have probabilies for class 1
2. sklearn_class_labels: this is the class label prediction computed by sklearn's density function
3. naive_class_labels: this is the class label prediction computed by using a naïve 0.5 threshold

In [6]:
predicted_probabilities = clf.predict_proba(X_test)
print('shape of predicted probabilties:', predicted_probabilities.shape)

shape of predicted probabilties: (188, 2)


In [7]:
sklearn_class_labels = clf.predict(X_test)
print('shape of model default class predicttions:', sklearn_class_labels.shape)

shape of model default class predicttions: (188,)


In [8]:
naive_class_labels = np.where(predicted_probabilities[:,1] > 0.5, 1, 0)
print('shape of naïve class predicttions:', naive_class_labels.shape)

shape of naïve class predicttions: (188,)


In [11]:
print('Recall score of sklearn predicted classes')
recall_score(sklearn_class_labels, y_test)

Recall score of sklearn predicted classes


0.9752066115702479

In [12]:
print('Recall score of naive predicted classes')
recall_score(naive_class_labels, y_test)

Recall score of naive predicted classes


0.9752066115702479

In [52]:
from sklearn.metrics import precision_recall_curve

precision, recall, threshold = precision_recall_curve(y_test, predicted_probabilities[:,1])
pr = pd.DataFrame({'threshold': threshold, 'precision': precision[:-1], 'recall': recall[:-1]})

In [57]:
pr

,threshold,precision,recall
0,0.212003,0.960317,1.000000
1,0.255675,0.960000,0.991736
2,0.289581,0.959677,0.983471
3,0.352528,0.959350,0.975207
4,0.484092,0.967213,0.975207
...,...,...,...
121,0.999457,1.000000,0.041322
122,0.999527,1.000000,0.033058
123,0.999543,1.000000,0.024793
124,0.999698,1.000000,0.016529


## Build Functions

In [63]:
import numpy as np
from sklearn.metrics import precision_recall_curve

class ThresholdOptimizer:
    def __init__(self, y_true, probas_pred, **kwargs):
        precision, recall, threshold = precision_recall_curve(y_true, probas_pred, **kwargs)
        self.best_precision_threshold = threshold[np.argmax(precision)-1]
        self.best_recall_threshold = threshold[::-1][np.argmax(recall[::-1])-1]

In [64]:
# example
optimizer = ThresholdOptimizer(y_test, predicted_probabilities[:,1])
optimizer.best_recall_threshold

0.2120027616991124